In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data_folder='./data/'

In [3]:
#RNA_seq data from  from The Cancer Genome Atlas pan-cancer dataset in "transcripts.txt"
example= pd.read_csv(data_folder +"transcripts.txt")

In [4]:
#load the network
def load_obj(name):
    with open(data_folder + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def entrez_to_name(entrezID):

    return gid2name[entrezID]

In [5]:
# all_Proteins.csv: contains mapping table between genes, transcripts and proteins generated from BioMart Ensembl
# g2d: dictionary of Pfam annotation of domains in genes
# gid2name: dictionary of gene id to gene symbol
# PPI and DDI networks from NetworkX
data = pd.read_csv(data_folder +"all_Proteins.csv")
PPI=load_obj("PPI")
DDI=load_obj("DDI")
gid2name=load_obj("gid2name")
g2d=load_obj("g2d")

In [6]:
def pr_to_tr(pr):
    df_filter = data['Protein stable ID'].isin([pr])
    try: return data[df_filter]['Transcript stable ID'].unique()[0]
    except IndexError:
        return False

def tr_to_gene(tr):
    df_filter = data['Transcript stable ID'].isin([tr])
    try: return data[df_filter]['Gene stable ID'].unique()[0]
    except IndexError:
        return False

def ensembl_to_entrez(gene):
    df_filter = data['Gene stable ID'].isin([gene])
    try: return data[df_filter]['NCBI gene ID'].unique()[0]
    except IndexError:
        return False
    
def entrez_to_ensembl(gene):
    df_filter = data['NCBI gene ID'].isin([gene])
    try: return data[df_filter]['Gene stable ID'].unique()[0]
    except IndexError:
        return False   
    
def tr_to_domain(tr):
    df_filter = data['Transcript stable ID'].isin([tr])
    tdata=data[df_filter]
    tdata=tdata[tdata["Pfam ID"].notna()].drop_duplicates()
    try: return tdata["Pfam ID"].unique()
    except IndexError:
        return False

#add to view
def check_PPI_status(tr):
    df_filter = data['Transcript stable ID'].isin([tr])
    return PPI.has_node(data[df_filter]['NCBI gene ID'].astype('str').unique()[0]) 


def tr_is_coding(tr):
    return  len(data[data['Transcript stable ID'].isin([tr])])!=0

def filter_proteins_list(List):
    filtred_list=[]
    for tr in List:
        #Correct spaces, dotsof version....
        ftr=tr=tr.replace(" ", "")
        ftr=ftr.split(".")[0]
        
        
        #make sure Correct ID
        if len(ftr)==15 and ftr[0:3]=='ENS':
        
            #Check if protein coding
            
            #input is a protein and coverted successfully 
            if ftr[3]=='P' :
                tmp= pr_to_tr(ftr)
                # The protein is converted to a transcript
                if tmp!=False:
                        # Check PPI status
                        if check_PPI_status(tmp):
                            filtred_list.append(tmp)
                
            #check if transcript is a coding protein
            elif ftr[3]=='T'and tr_is_coding(ftr) and check_PPI_status(ftr):
               filtred_list.append(ftr)
                
                
    return filtred_list


def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

In [7]:
# you can choose any sample from the RNA_seq data
sample_id='TCGA-S9-A7J2-01'

example=example.sort_values([sample_id], ascending=[False])


In [8]:
example

,sample,TCGA-S9-A7J2-01
125701,ENST00000508163.1,100.0
117953,ENST00000560688.1,100.0
27882,ENST00000313071.6,100.0
138098,ENST00000330689.5,100.0
84179,ENST00000542475.2,100.0
10345,ENST00000370862.3,100.0
84184,ENST00000469929.1,100.0
68009,ENST00000623894.1,100.0
48761,ENST00000560434.1,100.0
138083,ENST00000603875.1,100.0


In [9]:
#keep all expressed transcripts > threshold=1
# you can choose a different threshold by changing thi value
threshold=1
df_filter = example['TCGA-S9-A7J2-01']>=threshold
expressed_tr=example[df_filter]

In [10]:
expressed_tr

,sample,TCGA-S9-A7J2-01
125701,ENST00000508163.1,100.0
117953,ENST00000560688.1,100.0
27882,ENST00000313071.6,100.0
138098,ENST00000330689.5,100.0
84179,ENST00000542475.2,100.0
10345,ENST00000370862.3,100.0
84184,ENST00000469929.1,100.0
68009,ENST00000623894.1,100.0
48761,ENST00000560434.1,100.0
138083,ENST00000603875.1,100.0


In [11]:
expressed_tr=expressed_tr['sample'].unique()

In [12]:
expressed_tr

array(['ENST00000508163.1', 'ENST00000560688.1', 'ENST00000313071.6', ...,
       'ENST00000498176.1', 'ENST00000553939.5', 'ENST00000456829.6'],
      dtype=object)

In [13]:
filtred=filter_proteins_list(expressed_tr)

In [14]:
#all expressed isoforms
len(filtred)

30212

In [15]:
# get the Pfam domains
gene_domains={}
for tr in filtred:
    
    gene=tr_to_gene(tr)
    domains=tr_to_domain(tr)
 
    if len(domains)>0:
        #Check for every domain
        for domain in domains:

                if gene not in gene_domains:
                    
                    gene_domains[gene]=[domain]

                else :
                    gene_domains[gene].append(domain)
                    gene_domains[gene]=Remove(gene_domains[gene])

In [16]:
gene_domains

{'ENSG00000204427': ['PF00561'],
 'ENSG00000112159': ['PF17867', 'PF07728', 'PF17865'],
 'ENSG00000130592': ['PF02029'],
 'ENSG00000161944': ['PF00059', 'PF03954'],
 'ENSG00000160058': ['PF03909'],
 'ENSG00000232112': ['PF09072'],
 'ENSG00000134597': ['PF00076'],
 'ENSG00000115825': ['PF00169', 'PF00130', 'PF00069'],
 'ENSG00000006047': ['PF00313'],
 'ENSG00000170122': ['PF00250'],
 'ENSG00000050130': ['PF05571'],
 'ENSG00000162819': ['PF03097'],
 'ENSG00000141294': ['PF14580'],
 'ENSG00000173085': ['PF01040'],
 'ENSG00000125945': ['PF01352', 'PF00096'],
 'ENSG00000198835': ['PF00029'],
 'ENSG00000117640': ['PF05308'],
 'ENSG00000183808': ['PF00076'],
 'ENSG00000167554': ['PF00096', 'PF01352'],
 'ENSG00000096433': ['PF08709', 'PF01365', 'PF08454', 'PF00520', 'PF02815'],
 'ENSG00000189362': ['PF10225'],
 'ENSG00000273983': ['PF00125'],
 'ENSG00000142303': ['PF00090',
  'PF01562',
  'PF01421',
  'PF08686',
  'PF05986',
  'PF17771'],
 'ENSG00000163807': ['PF15377'],
 'ENSG00000100906': ['

In [17]:
#number of genes with known domains
len(gene_domains)

12866

In [18]:
missing={}
proteins_id=[]
for tr in filtred:
    ensembl=tr_to_gene(tr)
    entrez=str(ensembl_to_entrez(ensembl))
    proteins_id.append(entrez)
    if entrez in g2d:
        domains=g2d[entrez]
    
        
        #all domains are lost for this gene
        if ensembl not in gene_domains :
            missing[ensembl]=domains
            
        #check if all domains are there   
        else:
            missing[ensembl]=[]
            for d in domains:
                if d not in gene_domains[ensembl]:
                    missing[ensembl].append(d)
                    
                    
proteins_id=Remove(proteins_id)

In [19]:
#number of expressed genes 
print('number of expressed genes',len(proteins_id))
print('number of expressed isoforms',len(filtred))

number of expressed genes 13732
number of expressed isoforms 30212


In [20]:
H = PPI.subgraph(proteins_id)

In [21]:
print('number of interactions',len(H.edges()))

number of interactions 287753


In [22]:
#identify structurally annotated interactions
# detect affected edges and missing domains mediating an interaction

E=[]
N=[]
affected_nodes=[]
DDI_nodes=[]


p1=[]
p2=[]
p1_name=[]
p2_name=[]
DDIs=[]
missing_DDIs=[]
for e in H.edges():
    if e[0] not in N: N.append(e[0])
    if e[1] not in N: N.append(e[1])

    gene1=entrez_to_ensembl(e[0])
    gene2=entrez_to_ensembl(e[1])
    
    
    #check if gene is in missing dictionary
    #missing{} have all proteins with domain
    
    
    #Draw The PPI edge for every case
   
    #one of the gene have a domain
    if (gene1 in missing) and (gene2 in missing):
        inter=False
        domains1=g2d[e[0]]
        domains2=g2d[e[1]]
        
        DDIs_tmp=[]
        missing_DDIs_tmp=[]
        
        for d1 in domains1:
            for d2 in domains2:
                if DDI.has_edge(d1,d2):
                     inter=True
                        

                    
                     #Interacted domain is missing
                     if (d1 in missing[gene1]) or (d2 in missing[gene2]):
                            E.append("{from: '"+e[0]+"', to: '"+e[1]+"', dashes:  true,title:' Lost "+d1+'-'+d2+"', color: 'red'},") 
                            #print('yeaaaaah')
                            affected_nodes.append(e[0])
                            affected_nodes.append(e[1])
                            
                            missing_DDIs_tmp.append(d1+'-'+d2)
                     else:
                            #Interaction retained
                            E.append("{from: '"+e[0]+"', to: '"+e[1]+"',title:'"+d1+'-'+d2+"',  color: 'red'},") 
                            DDI_nodes.append(e[0])
                            DDI_nodes.append(e[1])  
                            DDIs_tmp.append(d1+'-'+d2)
                            
        if inter:
             # inter=True: a Domain-Domain Interaction was found:
             #for the table
             p1.append(gene1)
             p1_name.append(entrez_to_name(e[0]))  
             p2.append(gene2)
             p2_name.append(entrez_to_name(e[1]))
             if DDIs_tmp!=[]: DDIs.append(' ; '.join(DDIs_tmp))
             else: DDIs.append('-')
             if missing_DDIs_tmp!=[]: missing_DDIs.append(' ; '.join(missing_DDIs_tmp))
             else: missing_DDIs.append('-')
                        
                        

In [23]:
pd_interaction=pd.DataFrame( data=[p1,p1_name,p2,p2_name,DDIs,  missing_DDIs ], index=['Protein 1 ID', 'P1 name','Protein 2 ID', 'P2 name', 'Retained DDIs', 'Lost DDIs'] ) 
pd_interaction=pd_interaction.transpose()

In [24]:
pd.set_option('display.max_colwidth',1000)
data=pd_interaction

In [26]:
genes=list(data['Protein 1 ID'].unique())
genes=[genes.append(x) for x in data['Protein 2 ID'].unique() if x not in genes ]

In [27]:
affected_edges=data[data['Lost DDIs']!='-']

In [28]:
len(data),len(genes)

(41449, 1476)

In [29]:
#all PPI with missing domains
affected_edges

,Protein 1 ID,P1 name,Protein 2 ID,P2 name,Retained DDIs,Lost DDIs
42,ENSG00000132475,H3-3B,ENSG00000085224,ATRX,-,PF00125-PF00271 ; PF00125-PF00176
60,ENSG00000169398,PTK2,ENSG00000185532,PRKG1,-,PF07714-PF00069
63,ENSG00000169398,PTK2,ENSG00000145819,ARHGAP26,PF07714-PF14604,PF07714-PF00169
71,ENSG00000169398,PTK2,ENSG00000136205,TNS3,-,PF03623-PF00017 ; PF07714-PF00017
92,ENSG00000169398,PTK2,ENSG00000085733,CTTN,PF07714-PF14604,PF07714-PF00018
93,ENSG00000169398,PTK2,ENSG00000111859,NEDD9,PF07714-PF14604,PF07714-PF00018
127,ENSG00000094631,HDAC6,ENSG00000099995,SF3A1,-,PF02148-PF00240
140,ENSG00000094631,HDAC6,ENSG00000069399,BCL3,-,PF00850-PF00023
199,ENSG00000113575,PPP2CA,ENSG00000197363,ZNF517,-,PF00149-PF00096
210,ENSG00000113575,PPP2CA,ENSG00000108424,KPNB1,-,PF00149-PF02985


In [30]:
lost_edges=affected_edges[affected_edges['Retained DDIs']=='-']

In [31]:
#completly lost PPIs

lost_edges

,Protein 1 ID,P1 name,Protein 2 ID,P2 name,Retained DDIs,Lost DDIs
42,ENSG00000132475,H3-3B,ENSG00000085224,ATRX,-,PF00125-PF00271 ; PF00125-PF00176
60,ENSG00000169398,PTK2,ENSG00000185532,PRKG1,-,PF07714-PF00069
71,ENSG00000169398,PTK2,ENSG00000136205,TNS3,-,PF03623-PF00017 ; PF07714-PF00017
127,ENSG00000094631,HDAC6,ENSG00000099995,SF3A1,-,PF02148-PF00240
140,ENSG00000094631,HDAC6,ENSG00000069399,BCL3,-,PF00850-PF00023
199,ENSG00000113575,PPP2CA,ENSG00000197363,ZNF517,-,PF00149-PF00096
210,ENSG00000113575,PPP2CA,ENSG00000108424,KPNB1,-,PF00149-PF02985
221,ENSG00000113575,PPP2CA,ENSG00000175634,RPS6KB2,-,PF00149-PF00069
227,ENSG00000113575,PPP2CA,ENSG00000171940,ZNF217,-,PF00149-PF00096
233,ENSG00000113575,PPP2CA,ENSG00000178150,ZNF114,-,PF00149-PF00096


In [32]:
#save results
data.to_csv('./Results/3D_subnetwork.csv')